<a href="https://colab.research.google.com/github/DayeSong713/Healthcare_data/blob/main/Healthcare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install python3-dev default-limbmysqlclient-dev
!pip install pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package default-limbmysqlclient-dev
     |████████████████████████████████| 43 kB 1.0 MB/s 


In [ ]:
import pandas  as pd
import pymysql

In [ ]:
conn = pymysql.connect(host = '49.50.167.136',
                       port = 5432,
                       user = 'walker102',
                       password = 'forcebewithyou',
                       db='synthea_1000')

OperationalError: ignored

In [ ]:
cur=conn.cursor()


1. 첫 번째 쿼리는 모든 환자의 총 내원일수이며 두 번째 쿼리는 총 내원일수의 최대값과 총 내원일수 최대값을 가지는 환자의 수이다. 첫 번째 쿼리를 서브 쿼리로 이용해서 두 번째 쿼리를 작성했다.

In [ ]:
cur.execute("select person_id, sum(datediff(visit_end_date,visit_start_date)+1) as visit
               from visit_occurrence
               group by person_id")

rows=cur.fetchall()
print(rows)

In [ ]:
cur.execute("select datediff(visit_end_date,visit_start_date)+1, count(*)
               from visit_occurence
              where sum(datediff(visit_end_date,visit_start_date)+1) = (select max(sum(datediff(visit_end_date,visit_start_date)+1) as visit)
                                                                          from visit_occurrence
                                                                         group by person_id)
            group by person_id")

rows=cur.fetchall()
print(rows)

2. 먼저 concept_occurrence table과 concept table을 조인하고 a,b,c,d,e로 시작하는 상병의 이름을 찾기 위해 정규 표현식을 사용했다. heart 가 들어가는 상병명을 찾기 위해 like를 사용했다. 

In [ ]:
cur.execute("select distinct c.concept_name
               from condition_occurrence o
              inner join concept c on c.concept_id=o.condition_concept_id
              where c.concept_name regexp '^[abcde]' and  
                    c.concpet_name like "%heart%"
                      ")
rows=cur.fetchall()
print(rows)

3. where 절에 환자번호 조건을 주고 마지막 종료일과 처음 시작일 컬럼으로 연산해 복용일을 구했다.

In [ ]:
cur.execute("select drug_concept_id, drug_exposure_start_datetime, drug_exposure_end_datetime,  datediff(visit_end_date,visit_start_date) as period
               from drug_exposure
              where person_id=1891866
              order by period desc")
rows=cur.fetchall()
print(rows)      

4.3개의 테이블을 조인하되, pair table의 두 약 모두 처방 건수를 알아야 하므로 prescription_count를 각각 첫 번째 약과 두 번째 약을 기준으로 하여 두 번 조인했다. 약품명은 첫 번째 약만 알면 되므로 drugs는 1번만 조인했다. 첫 번째 약의 처방 건수보다 두 번째 약의 처방 건수가 높다는 조건을 주어 데이터를 필터링하고, 처방 건수별로 많은 것부터 정렬했다.

In [ ]:
cur.execute("with drug_list as(
select distinct drug_concept_id, concept_name, count(*) as cnt from synthea_cdm_1000.drug_exposure de join synthea_cdm_1000.concept on drug_concept_id=concept_id
where concept_id in (40213154,19078106,19009384,40224172,19127663,1511248,40169216,1539463,
19126352,1539411,1332419,40163924,19030765,19106768,19075601)
group by drug_concept_id,concept_name
order by count(*) desc
)
, drugs as (select drug_concept_id, concept_name from drug_list)

select d.concept_name
from drug_pair dp
inner join drugs d on dp.drug_concept_id1= dp.drug_concept_id 
inner join prescription_count p1 on dp.concept_id1=p.drug_concept_id
inner join prescription_count p2 on 
dp.concept_id2=p.drug_concept_id
where p2.count  > p1.count
order by p1.count desc
")
rows=cur.fetchall()
print(rows)

5.당뇨병 진단, 환자의 나이, 약물 복용에 대한 정보가 모두 필요했기 때문에 person table, condition_occurrence table, drug_exposure table 3개를 조인했다. 
제2형 당뇨병, 18세 이상, metformin 복용일을 조건으로 주어 데이터를 필터링하고, 전체 환자수를 추출했다.

In [ ]:
cur.execute("select count(*)
from person p
inner join condidion_occurrence c on p.person_id=c.person id
inner join drug_exposure d on p.person_id=d.person_id

where c.condition_concept_id in (3191208,36684827,3194332,3193274,43531010,4130162,45766052,
45757474,4099651,4129519,4063043,4230254,4193704,4304377,20
1826,3194082,3192767) and
p.birth_date < date_sub(now(), interval 18 year)
and d.drug_concept_id=40163924 and
datediff(drug_exposure_end_date, drug_exposure_start_data) >= 90
")
rows=cur.fetchall()
print(rows)

6. 제2형당뇨를 진단받은 환자 중 4개의 의약품을 처방받은 환자의 정보까지는 추출했지만, 의약품 처방 패턴은 추출하지 못했다.

In [ ]:
cur.execute("select drug_concept_id
  from condition_occurrence c
inner join drug_exposure d on c.person_id=d.person_id
where c.condition_concept_id in (3191208,36684827,3194332,3193274,43531010,4130162,45766052,
45757474,4099651,4129519,4063043,4230254,4193704,4304377,20
1826,3194082,3192767)
and d.drug_concept_id in (19018935,1539411,1539463, 19075601, 11151171 )
group by c.person_id
")
rows=cur.fetchall()
print(rows)

In [ ]:
conn.close()